In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-06-29 17:13:08--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.68MB/s    in 0.2s    

2020-06-29 17:13:08 (4.68 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmznReviewsETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
sports_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"),sep='\t', header=True, inferSchema=True)

sports_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...|2015-08-31 00:00:00|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...| 

In [ ]:
# Finding the number of records in the dataset
sports_data_df.count()

4850360

In [ ]:
sports_data_cleaned_df = sports_data_df.dropna()
sports_data_cleaned_df.count()

4848999

In [ ]:
# counting the distinct customers

customers_data = sports_data_cleaned_df.groupBy("customer_id").count()
customers_data.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   35669025|    1|
|   48198949|   30|
|   43355824|    6|
|   33014261|    6|
|   23493243|    1|
|   30717305|    4|
|   15714077|    2|
|    7854719|    1|
|   12761428|    2|
|   14127895|    1|
|   51451778|    4|
|    3452965|    1|
|   40430762|    2|
|   27314089|   15|
|   11405991|    1|
|   20368048|    2|
|   33967841|    2|
|   10418855|    1|
|   43783459|    1|
|   50843047|    4|
+-----------+-----+
only showing top 20 rows



In [ ]:
# renaming the 'count' column
customers_data = customers_data.withColumnRenamed("count","customer_count")
customers_data.show()
type(customers_data)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35669025|             1|
|   48198949|            30|
|   43355824|             6|
|   33014261|             6|
|   23493243|             1|
|   30717305|             4|
|   15714077|             2|
|    7854719|             1|
|   12761428|             2|
|   14127895|             1|
|   51451778|             4|
|    3452965|             1|
|   40430762|             2|
|   27314089|            15|
|   11405991|             1|
|   20368048|             2|
|   33967841|             2|
|   10418855|             1|
|   43783459|             1|
|   50843047|             4|
+-----------+--------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [ ]:
customers_data.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
# changing the datatype of customer_count to interger
customers_data= customers_data.withColumn("customer_count", customers_data.customer_count.cast("int"))

In [ ]:
customers_data.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
# Create products dataframe 
products_data = sports_data_cleaned_df.groupBy('product_id','product_title').count()
products_data = products_data.select(['product_id','product_title'])

products_data.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0092702VE|Abu Garcia Revo S...|
|B00HZ9WTVC|Lee 90355 Cutter ...|
|B00WS4FRQ8|BEEWARE Targets -...|
|B00AKJVYYC|NEW Callaway RAZR...|
|B00DIPSOM0|GoSports Ladder T...|
|B0017LEUCU|Body Sport Exerci...|
|B00B59OMS6|Swiftwick SUSTAIN...|
|B00KC5SYLU|EliteTek RG-14 Fo...|
|B000GKUAB8|Multisport Helmet...|
|B001P80Z10|Izzo Golf Mini Mo...|
|B0063HVE18|Weston 30-0008-W ...|
|B00UR4HHDM|  Mission Multi-Cool|
|B000P3WQ86|Plano Pro-Max Sin...|
|B00005UDHY|Speedo Hydrospex ...|
|B00NZCXMUA|Fastfender Sail F...|
|B000YPIENQ|5.11 Tactical #73...|
|B00I3P5PM2|Tfo Gts Inshore G...|
|B009SPBB10|Twins Special Ful...|
|B00WIWYBZI|Black Anime Foam ...|
|B004SRA7GI|Moultrie Game Spy...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
products_data.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# Create review_id_table dataframe 
review_id_table = sports_data_cleaned_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130|2015-08-31 00:00:00|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679|2015-08-31 00:00:00|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336|2015-08-31 00:00:00|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227|2015-08-31 00:00:00|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588|2015-08-31 00:00:00|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483|2015-08-31 00:00:00|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496|2015-08-31 00:00:00|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265|2015-08-31 00:00:00|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165|2015-08-31 00:00:00|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713|2015-08-31 00:00:00|
|R31673RTGEZSW7|   346576

In [ ]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# changing the datatype of review_date to date
review_id_table=review_id_table.withColumn("review_date", review_id_table.review_date.cast("date"))

In [ ]:
# Create vine_table dataframe 
vine_table = sports_data_cleaned_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|
|R32M0YEWV77XG8|          5|            1|          1|   N|
| RR8V7WR27NXJ5|          1|            0|          0|   N|
|R1MHO5V9Z932AY|          5|            0|          0|   N|
|R16PD71086BD2V|          5|            0|          1|   N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|
|R3AUMSHAW73HWN|          4|            2|          3|   N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|
|R3H9543FWBWFBU|          5|            1|          1|   N|
| RUANXOQ9W3OU5|          5|            0|          0|   N|
|R31673RTGEZSW7|          5|            2|          2|   N|
|R22OQLFSH42RCM|          5|            1|          1|   N|
|R12LEL4F3TSZUJ|          5|            2|          2|   N|
|R2L9XWD03072NI|          5|            

In [ ]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
# changing the datatype of star_rating to integer
vine_table=vine_table.withColumn("star_rating", vine_table.star_rating.cast("int"))

In [ ]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql:// :5432/Amzn_reviews"
config = {"user":<user>, 
          "password":<password>, 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to customers table in RDS

customers_data.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)



In [ ]:
sc= spark.sparkContext
from pyspark.sql import SQLContext
sqlContext= SQLContext(sc)
sqlContext.read.jdbc(url=jdbc_url, table='products', properties=config)

DataFrame[product_id: string, product_title: string]

In [ ]:
# Write DataFrame to products table in RDS

products_data.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to review_id_table  in RDS

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table  in RDS

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)